In [1]:
import pyopencl as cl
import pyopencl.array as cl_array
import numpy
import numpy.linalg as la
from pyopencl.elementwise import ElementwiseKernel

In [2]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [3]:
a_gpu = cl_array.to_device(queue,(numpy.random.randn(50000)+1j*numpy.random.randn(50000)).astype(numpy.complex64))
b_gpu = cl_array.to_device(queue,(numpy.random.randn(50000)+1j*numpy.random.randn(50000)).astype(numpy.complex64))

In [4]:
complex_prod = ElementwiseKernel(context = ctx,
        arguments = "float a, cfloat_t *x, cfloat_t *y, cfloat_t *z",
        operation = "z[i] = cfloat_rmul(a, cfloat_mul(x[i], y[i]))",
        name = "complex_prod",
        preamble = "#include <pyopencl-complex.h>")

In [5]:
real_part = ElementwiseKernel(context = ctx,
        arguments = "cfloat_t *x, float *z",
        operation = "z[i] = cfloat_real(x[i])",
        name = "real_part",
        preamble = "#include <pyopencl-complex.h>")

In [6]:
# 常数*复数*复数
c_gpu = cl_array.empty_like(a_gpu)
complex_prod(5, a_gpu, b_gpu, c_gpu)

In [7]:
# 获取上面计算结果的实部
c_gpu_real = cl_array.empty(queue, len(a_gpu), dtype=numpy.float32)
real_part(c_gpu, c_gpu_real)
print(c_gpu.get().real - c_gpu_real.get())

[0. 0. 0. ... 0. 0. 0.]


In [8]:
# CPU验证
print(la.norm(c_gpu.get() - (5*a_gpu.get()*b_gpu.get())))
assert la.norm(c_gpu.get() - (5*a_gpu.get()*b_gpu.get())) < 1e-3

0.0001351835
